# Import Libraries

In [2]:
from selenium import webdriver 
from bs4 import BeautifulSoup
from time import sleep
import sqlite3
from sqlite3 import Error

# SQLite Functions

In [3]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [4]:
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [5]:
def create_linkedin_profiles_table(conn):
    sql_create_linkedin_profiles_table = """ CREATE TABLE IF NOT EXISTS linkedin_profiles (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        headline text
                                    ); """
    if conn is not None:
        create_table(conn, sql_create_linkedin_profiles_table)

    else:
        print("Error! cannot create the database connection.")

In [11]:
def insert_linkedin_profile(conn, profile):
    sql = ''' INSERT INTO linkedin_profiles(name,headline)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, profile)
    conn.commit()
    return cur.lastrowid

In [12]:
def close_connection(conn):
    conn.close()

# Set Settings And Variables

In [13]:
PATH = "C:\\Program Files (x86)\\chromedriver.exe" 
driver = webdriver.Chrome(PATH) 
profile_counter = 1
conn = create_connection(r"C:\Users\root\Desktop\pythonsqlite.db")
create_linkedin_profiles_table(conn)

# Logfile

In [14]:
def open_logfile(path):
    return open(path,'a',encoding='utf-8')

def insert_to_logfile(file,id,profile):
    file.write(f"{id},\"{profile[0]}\",\"{profile[1]}\"\n")
        
def close_logfile(file):
    file.close()